# Unsloth: Mental Health Chatbot Fine-tuning

This notebook demonstrates how to fine-tune a model to become a **Mental Health Chatbot**.

We will use the `Amod/mental_health_counseling_conversations` dataset and format it using a standard chat template.

In [ ]:
%%capture
!pip install unsloth
!pip install --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit", # Instruct model is good for chat
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

### LoRA Configuration

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

### Data Preparation
We use the `Amod/mental_health_counseling_conversations` dataset. We need to format the `Context` (User) and `Response` (Assistant) into a chat format.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations", split = "train")

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Use Llama 3 template
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
)

def formatting_prompts_func(examples):
    convos = examples["Context"]
    responses = examples["Response"]
    texts = []
    for convo, response in zip(convos, responses):
        # Create a standard conversation structure
        conversation = [
            {"role": "user", "content": convo},
            {"role": "assistant", "content": response},
        ]
        text = tokenizer.apply_chat_template(conversation, tokenize = False, add_generation_prompt = False)
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer.train()

### Inference
Let's test our mental health chatbot.

In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "I feel very anxious about my job interview tomorrow."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)